# Implementing Neural Network Training with Different Optimizers

### Objective:
The goal of this assignment is to implement a small neural network from scratch and train it using three different optimization algorithms: Stochastic Gradient Descent (SGD), SGD with Momentum, and SGD with AdaGrad. You will need to compare the performance of these optimizers.

## 1- Dataset

### Digits

This dataset, sourced from sklearn, consists of 1797 images, each sized 8x8 pixels. Every image, like the example shown below, depicts a handwritten digit. To utilize an 8x8 image, it must first be transformed into a feature vector of length 64.

The task with this dataset is to classify each digit, with a total of 10 classes.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

digits = load_digits()
print("dataset's dimensions:", digits.data.shape)
plt.gray()
plt.matshow(digits.images[0])
plt.show()


In [ ]:
# this function loads train and test sets from digits dataset
def load_data():
    digits = load_digits()
    X = digits.data
    y = digits.target

    enc = OneHotEncoder()
    y = enc.fit_transform(y.reshape(-1, 1)).toarray()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, y_train, X_test, y_test

## 2- Neural Network Architecture:
* Implement a simple feedforward neural network with one hidden layer.
* Use sigmoid activation for the hidden layer and softmax for the output layer.
* Feel free to add any necessary functions to the class below.
* See figure 2 in assignment pdf for a diagram

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def sigmoid_derivative(s):
    return s * (1 - s)


def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)


class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

        self.dW1 = np.zeros((input_size, hidden_size))
        self.db1 = np.zeros((1, hidden_size))
        self.dW2 = np.zeros((hidden_size, output_size))
        self.db2 = np.zeros((1, output_size))

    def forward(self, X):
        # Implement forward pass
        # You should store the intermediate values as fields to be used by the backward pass
        # The forward pass must work for all batch sizes

        # X: the x values, batched along dimension zero
        # return: z2
        self.a1 = np.dot(X, self.W1) + self.b1
        self.z1 = sigmoid(self.a1)
        self.a2 = np.dot(self.z1, self.W2) + self.b2
        self.z2 = softmax(self.a2)
        return self.z2

    def backward(self, X, y):
        # Implement backward pass, assuming the MSE loss
        # Use the intermediate values calculated from the forward pass,
        # and store the gradients in fields
        # The backward pass only need to work for batch size 1

        # X: the x values, batched along dimension zero
        # y: batched target values
        # return: None
        d2 = (self.z2 - y) / y.shape[0]

        # Gradients for W2 and b2
        self.dW2 = np.dot(self.z1.T, d2)
        self.db2 = np.sum(d2, axis=0, keepdims=True)

        d1 = np.dot(d2, self.W2.T) * sigmoid_derivative(self.z1)

        # Gradients for W1 and b1
        self.dW1 = np.dot(X.T, d1)
        self.db1 = np.sum(d1, axis=0, keepdims=True)

    def get_params_and_grads(self):
        # Return parameters and corresponding gradients
        params = [self.W1, self.b1, self.W2, self.b2]
        grads = [self.dW1, self.db1, self.dW2, self.db2]
        return params, grads

## 3- Helper Functions
Implement the following helper functions (the "plot_all_results" function is already implemented):

* "mean_squared_error(predictions, targets)": This function receives the predicted values from the network and the target values, calculates the mean squared loss, and returns the loss value.

* "compute_accuracy(predictions, targets)": This function receives the predicted values and the target values, and returns the accuracy of the predictions.

* Feel free to add any other necessary helper functions here.


In [ ]:
def mean_squared_error(predictions, targets):
    return np.mean((predictions - targets) ** 2)


def compute_accuracy(predictions, targets):
    pred_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(targets, axis=1)
    return np.mean(pred_classes == true_classes)


def plot_all_results(all_losses, all_accuracies, all_labels):
    if len(all_losses) != len(all_accuracies):
        raise ValueError(
            "all_losses length must be equal to all_accuracies length")

    if len(all_losses) != len(all_labels):
        raise ValueError(
            "all_labels length must be equal to all_losses length")

    epochs = len(all_losses[0])
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    for i in range(len(all_losses)):
        plt.plot(range(1, epochs + 1), all_losses[i], label=all_labels[i])
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'Loss behaviours')

    plt.legend()

    plt.subplot(1, 2, 2)
    for i in range(len(all_losses)):
        plt.plot(range(1, epochs + 1), all_accuracies[i], label=all_labels[i])
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title(f'Accuracy behaviours')

    plt.legend()
    plt.show()

## 4- Optimizer Implementations:
Implement the following optimization algorithms:

 * Stochastic Gradient Descent (SGD)
 * SGD with Momentum
 * SGD with AdaGrad

In [ ]:
# Optimizer implementations (SGD, SGD with Momentum, AdaGrad)

class SGD():
    def __init__(self, params, learning_rate):
        self.params = params
        self.lr = learning_rate

    def step(self, grads):
        # Perform one step of SGD
        for param, grad in zip(self.params, grads):
            param -= self.lr * grad


class SGD_Momentum():
    def __init__(self, params, learning_rate, alpha):
        self.params = params
        self.lr = learning_rate
        self.alpha = alpha
        self.velocity = [np.zeros_like(param) for param in params]

    def step(self, grads):
        # Perform one step of SGD with momentum
        for i, (param, grad) in enumerate(zip(self.params, grads)):
            self.velocity[i] = self.alpha * self.velocity[i] - self.lr * grad
            param += self.velocity[i]


class SGD_AdaGrad():
    def __init__(self, params, learning_rate, delta):
        self.params = params
        self.lr = learning_rate
        self.delta = delta
        self.r = [np.zeros_like(param) for param in params]

    def step(self, grads):
        # Perform one step of SGD with adagrad
        for i, (param, grad) in enumerate(zip(self.params, grads)):
            self.r[i] += grad ** 2
            param -= self.lr * grad / (np.sqrt(self.r[i]) + self.delta)

## 5- Training Loop:

Use the following training function to train your neural network.


In [ ]:
# batch generator
def gen_batches(data, labels, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i+batch_size], labels[i:i+batch_size]


# Training loop
def train(network, data, optimizer, epochs, batch_size):
    X_train, y_train, X_test, y_test = data
    test_losses = []
    test_accuracies = []

    for epoch in range(epochs):
        random_indices = np.random.permutation(list(range(X_train.shape[0])))
        X_train = X_train[random_indices]
        y_train = y_train[random_indices]
        for x, y in gen_batches(X_train, y_train, batch_size):
            # Forward pass
            output = network.forward(x)

            # Backward pass
            network.backward(x, y)

            # Get parameters and gradients
            params, grads = network.get_params_and_grads()

            # Update parameters using the chosen optimizer
            optimizer.step(grads)

        # Compute loss and accuracy
        X_test = X_train
        y_test = y_train
        output = network.forward(X_test)
        train_loss = mean_squared_error(output, y_test)
        train_accuracy = compute_accuracy(output, y_test)

        test_losses.append(train_loss)
        test_accuracies.append(train_accuracy)

        print(
            f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    return test_losses, test_accuracies

## 6- Main function
Use the following main function to train your neural network based on sgd, sgd-momentum, and AdaGrad. The following hyperparameters are used in training:

* batch_size = 128
* input_size = 64
* hidden_size = 20
* output_size = 10
* learning_rate = 0.005
* epochs = 100

In [ ]:
# Main function to run the experiments
def main():
    data = load_data()
    batch_size = 128
    input_size = 64  # For the digits dataset (8x8 images flattened)
    hidden_size = 20
    output_size = 10
    learning_rate = 0.005
    epochs = 100

    print("Training with SGD LR 0.005")
    network = NeuralNetwork(input_size, hidden_size, output_size)
    test_losses_sgd, test_accuracies_sgd = train(network, data, SGD(
        network.get_params_and_grads()[0], learning_rate), epochs, batch_size)

    print("Training with SGD LR 0.05")
    network = NeuralNetwork(input_size, hidden_size, output_size)
    test_losses_sgd_b, test_accuracies_sgd_b = train(network, data, SGD(
        network.get_params_and_grads()[0], learning_rate * 10), epochs, batch_size)

    print("Training with SGD with Momentum LR 0.0005")
    network = NeuralNetwork(input_size, hidden_size, output_size)
    test_losses_momentum, test_accuracies_momentum = train(network, data, SGD_Momentum(
        network.get_params_and_grads()[0], learning_rate / 10, 0.9), epochs, batch_size)

    print("Training with AdaGrad LR 0.005")
    network = NeuralNetwork(input_size, hidden_size, output_size)
    test_losses_adagrad, test_accuracies_adagrad = train(network, data, SGD_AdaGrad(
        network.get_params_and_grads()[0], learning_rate, 1e-8), epochs, batch_size)

    print("Training with AdaGrad LR 0.05")
    network = NeuralNetwork(input_size, hidden_size, output_size)
    test_losses_adagrad_b, test_accuracies_adagrad_b = train(network, data, SGD_AdaGrad(
        network.get_params_and_grads()[0], learning_rate * 10, 1e-8), epochs, batch_size)

    # Compare train losses and train accuracies
    all_losses = [test_losses_sgd, test_losses_sgd_b,
                  test_losses_momentum, test_losses_adagrad, test_losses_adagrad_b]
    all_accuracies = [test_accuracies_sgd, test_accuracies_sgd_b,
                      test_accuracies_momentum, test_accuracies_adagrad, test_accuracies_adagrad_b]
    all_labels = ["SGD LR 0.005", "SGD LR 0.05",
                  "SGD-Momentum LR 0.0005", "AdaGrad LR 0.005", "AdaGrad LR 0.05"]
    plot_all_results(all_losses, all_accuracies, all_labels)


if __name__ == "__main__":
    main()

## 7. Increasing Layers

Instead of using 2-layer neural networks. We can increase the layer size to three and observe the loss again based on sgd. The newly added layer should adopt the same hidden size.

We are sticking to the same hyper-parameters as before.

* Plot the loss and accuracy.

* Discuss the impact of the increased layers in the pdf

In [ ]:
# Please implement the 3-layer neural network and run the main function again to plot the curve.
def relu(z):
    return np.maximum(0, z)


def relu_derivative(s):
    return np.where(s > 0, 1, 0)


class NeuralNetwork:
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        # Initialize weights and biases
        self.W1 = np.random.randn(
            input_size, hidden_size1) * np.sqrt(2. / input_size)
        self.b1 = np.zeros((1, hidden_size1))
        self.W2 = np.random.randn(
            hidden_size1, hidden_size2) * np.sqrt(2. / hidden_size1)
        self.b2 = np.zeros((1, hidden_size2))
        self.W3 = np.random.randn(
            hidden_size2, output_size) * np.sqrt(2. / hidden_size2)
        self.b3 = np.zeros((1, output_size))

        self.dW1 = np.zeros((input_size, hidden_size1))
        self.db1 = np.zeros((1, hidden_size1))
        self.dW2 = np.zeros((hidden_size1, hidden_size2))
        self.db2 = np.zeros((1, hidden_size2))
        self.dW3 = np.zeros((hidden_size2, output_size))
        self.db3 = np.zeros((1, output_size))

    def forward(self, X):
        # Implement forward pass
        self.a1 = np.dot(X, self.W1) + self.b1
        self.z1 = relu(self.a1)
        self.a2 = np.dot(self.z1, self.W2) + self.b2
        self.z2 = relu(self.a2)
        self.a3 = np.dot(self.z2, self.W3) + self.b3
        self.z3 = softmax(self.a3)
        return self.z3

    def backward(self, X, y):
        d3 = (self.z3 - y) / y.shape[0]

        # Gradients for W3 and b3
        self.dW3 = np.dot(self.z2.T, d3)
        self.db3 = np.sum(d3, axis=0, keepdims=True)

        d2 = np.dot(d3, self.W3.T) * relu_derivative(self.z2)

        # Gradients for W2 and b2
        self.dW2 = np.dot(self.z1.T, d2)
        self.db2 = np.sum(d2, axis=0, keepdims=True)

        d1 = np.dot(d2, self.W2.T) * relu_derivative(self.z1)

        # Gradients for W1 and b1
        self.dW1 = np.dot(X.T, d1)
        self.db1 = np.sum(d1, axis=0, keepdims=True)

    def get_params_and_grads(self):
        # Return parameters and corresponding gradients
        params = [self.W1, self.b1, self.W2, self.b2, self.W3, self.b3]
        grads = [self.dW1, self.db1, self.dW2, self.db2, self.dW3, self.db3]
        return params, grads


def main():
    data = load_data()
    batch_size = 128
    input_size = 64
    hidden_size1 = 20
    hidden_size2 = 20
    output_size = 10
    learning_rate = 0.005
    epochs = 100

    print("Training with SGD LR 0.005")
    network = NeuralNetwork(input_size, hidden_size1,
                            hidden_size2, output_size)
    test_losses_sgd, test_accuracies_sgd = train(network, data, SGD(
        network.get_params_and_grads()[0], learning_rate), epochs, batch_size)

    print("Training with SGD LR 0.05")
    network = NeuralNetwork(input_size, hidden_size1,
                            hidden_size2, output_size)
    test_losses_sgd_b, test_accuracies_sgd_b = train(network, data, SGD(
        network.get_params_and_grads()[0], learning_rate * 10), epochs, batch_size)

    print("Training with SGD with Momentum LR 0.0005")
    network = NeuralNetwork(input_size, hidden_size1,
                            hidden_size2, output_size)
    test_losses_momentum, test_accuracies_momentum = train(network, data, SGD_Momentum(
        network.get_params_and_grads()[0], learning_rate / 10, 0.9), epochs, batch_size)

    print("Training with AdaGrad LR 0.005")
    network = NeuralNetwork(input_size, hidden_size1,
                            hidden_size2, output_size)
    test_losses_adagrad, test_accuracies_adagrad = train(network, data, SGD_AdaGrad(
        network.get_params_and_grads()[0], learning_rate, 1e-8), epochs, batch_size)

    print("Training with AdaGrad LR 0.05")
    network = NeuralNetwork(input_size, hidden_size1,
                            hidden_size2, output_size)
    test_losses_adagrad_b, test_accuracies_adagrad_b = train(network, data, SGD_AdaGrad(
        network.get_params_and_grads()[0], learning_rate * 10, 1e-8), epochs, batch_size)

    # Compare train losses and train accuracies
    all_losses = [test_losses_sgd, test_losses_sgd_b,
                  test_losses_momentum, test_losses_adagrad, test_losses_adagrad_b]
    all_accuracies = [test_accuracies_sgd, test_accuracies_sgd_b,
                      test_accuracies_momentum, test_accuracies_adagrad, test_accuracies_adagrad_b]
    all_labels = ["SGD LR 0.005", "SGD LR 0.05",
                  "SGD-Momentum LR 0.0005", "AdaGrad LR 0.005", "AdaGrad LR 0.05"]
    plot_all_results(all_losses, all_accuracies, all_labels)


if __name__ == "__main__":
    main()